# FAQ

## Frequently Asked Questions

Below is a list of frequently asked questions, although these are not necessasily SPM12w specfic, I have attempted to answer them with an eye towards the available options and functions in SPM12.

### What are my options for dealing with motion?

There are four ways to deal with motion within the context of what is available in SPM12w:

1. Do nothing but realignment (p.realign = 1, p.unwarp = 0, glm.move = 0)
1. [Unwarp](http://www.fil.ion.ucl.ac.uk/spm/toolbox/unwarp) to correct for Movement by Susceptibility Interactions (p.realign = 1, p.unwarp = 1, glm.move = 0)
1. Include the realignment parameters as motion covariates in the GLM (p.realign = 1, p.unwarp = 0, glm.move = 1)
1. Censor high-movement "outlier" timepoints using Art Outlier Detection (p.realign = 1, p.unwarp = 0, glm.move = 0, glm.outliers = 1). 

```{note}
When _censoring_ high-movement timepoints, you will often want to grab one or two volumes before and after the motion spike as the effects of the spike will be spread out in time by slicetime correction.
```

The norm that we've been applying is to unwarp and include movement regressors in the GLM model, although it has been argued that this may be overkill as including the motion covariates in addition to unwarping is redundant. Generally, in cases of reasonable motion, it may be sufficient to simply perform unwarping and not include the realignment parameters as covariates in the GLM.

Indeed, it has been argued that this could have a detrimental effect on sensitivity if small movements correlates with the task. See Johnstone's [paper](https://onlinelibrary.wiley.com/doi/abs/10.1002/hbm.20219) on the pitfalls of always including realignment parameters.

### How should I filter or detrend my data for low frequency noise?

FMRI data typically contains a high degree of low-frequency "noise". Sources of this noise vary from physiological signals (e.g., respiration) to scanner signal drift. Different analysis packages have implemented a number of methods for "filtering" out this LF noise. Some packages employ temporal filtering prior to GLM modeling, effectively creating a new dataset in which LF noise has been removed. However, most packages these days appear to model the LF noise at the GLM phase. SPM used to include a series of cosine regressors in the model (discreet cosine matrix or DCT) which capture variance associated with low frequency components of the timeseries. Later versions instead filtered the entire design matrix through the DCT, effectively resulting in a model that will filter out low frequencies, but without losing any degrees of freedom due to the inclusion of additional regressors. AFNI, on the other hand, typically accounts for low frequency trends in the data by including a series of polynomial regressors in the GLM. The simplest of these (-polort 1) is a constant plus a linear trend regressors. 

```{note}
It is possible to caompute the analogous high-pass filter cutoff corresponding to the polynomial regressors included in a GLM using the following formula. As per the AFNI help for 3dDeconvolve: _"For pnum > 2, this type of baseline detrending is roughly equivalent to a highpass filter with a cutoff of (p-2)/D Hz, where 'D' is the duration of the imaging run: D = N*TR"_
```

Within the context of SPM12w, we've typically only included a constant and linear trend term (per session) rather than use SPM's DCT method. The primary reason for this is that, unlike traditional SPM analyses, we concatenate events into a single regressor per condition (i.e., we concatenate runs). It was generally beleived that the DCT method used by SPM wasn't valid when concatenating runs due to it being blind to the breaks in the timeseries.

However, at a MGH workshop I participated in, Dan Gittelman suggested this wasn't a big deal and it was fine to use SPM's traditional high pass filter on concatenated data. Nevertheless, with SPM12w, we opted to follow the same conceptual method as AFNI by using a 1st order polynomial to model low frequency trends in the data (i.e. a linear trend regressor per session).

Starting with SPM8w (and continued into SPM12w), we've added the option to include more polynomials. Specifically, the parameter __p.polyord__ specifies the order of the polynomials to include in the design. 

* p.polyord = 0 models only the session mean, and does not model trends at all (not recommended). 
* p.polyord = 1 Includes only the linear term and is likely sufficient for short to medium length runs.
* p.polyord = 2 or 3 adds 2nd and/or 3rd order polynomials to the model (i.e., quadratic and cubic trends). 

```{tip}
Increasing the number of polynomials beyond the quadtratic and cubic trends is allowed in SPM12w but it is generally not necessary [see paper](http://www.sciencedirect.com/science/article/pii/S1053811902910530). 

```

Below are three plots demonstrating the timeseries and power spectra for one voxel (MNI: -21,-6,-21) of one run of 268 TRs, 3 Conditions + 2 Instruction Conditions (on/off cues) and 6 motion covariates. This plot was made with _spm12w_timeplot.m_. The opaque band in the Frequency domain graphs highlights the portion of the frequency spectrum typically filtered out by using SPM's default 128second filter. For this voxel, there’s not much power in the low-frequency range to worry about, even without filtering.


Raw data for a single voxel prior to GLM modeling mean-centered only (i.e., __p.polyord = 0__):

```{figure} ../assets/ch6_timeplot1.png
---
width: 550px
name: directive-fig1
---
Figure 1: Raw data for a single voxel prior to GLM (mean centered only)
```

This figure shows the power spectrum for the same timeseries with linear trend removal:

```{figure} ../assets/ch6_timeplot2.png
---
width: 550px
name: directive-fig2
---
Figure 2: Timeseries after linear trend removal.
```
 
And here’s the same timeseries with linear and quadratic trend removal (i.e., __p.polyord = 2__). Low-frequency noise is almost as good as SPM's 128s HPF and the time series itself is flatter. One could include the cubic trends (i.e., __p.polyord = 3__), but in my tests this did not show any improvement above modeling linear and quadratic trends, thought it may matter with longer runs.
  
```{figure} ../assets/ch6_timeplot3.png
---
width: 550px
name: directive-fig3
---
Figure 3: Timeseries after linear and quadratic trend removal.
```